In [21]:
import numpy as np
import pandas as pd

In [34]:
ratings_df = pd.read_csv('./Data/merged_data.csv')
ratings_df.insert(0, 'User_ID', range(1, len(ratings_df) + 1))
ratings_df = ratings_df.rename(columns={'0': 'Jokes_Rated'})
ratings_df.to_csv('./Data/structured.csv', index=False)
ratings_df.head()

,User_ID,Jokes_Rated,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,1,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,3,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,4,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,5,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [23]:
def replace_99(ratings):
    joke_ids = ratings.columns[2:]    
    for joke_id in joke_ids: 
        ratings[joke_id] = ratings[joke_id].replace([99],np.nan)
    return ratings
def normalization(ratings):
    total_users = ratings.shape[0]
    for i in range(total_users):
        ratings.iloc[i, 2:] = ratings.iloc[i, 2:] - np.mean(ratings.iloc[i, 2:])
    return ratings
def replace_0(ratings):
    joke_ids = ratings.columns[2:]    
    for joke_id in joke_ids: 
        ratings[joke_id] = ratings[joke_id].replace([np.nan],0)
    return ratings

In [24]:
normalized_ratings = replace_99(ratings_df)
normalized_ratings = normalization(normalized_ratings)
normalized_ratings = replace_0(normalized_ratings)
normalized_ratings.to_csv('./Data/normalized.csv', index=False)
normalized_ratings.head()

,User_ID,Jokes_Rated,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,1,74,-4.388108,12.221892,-6.228108,-4.728108,-4.088108,-5.068108,-6.418108,7.601892,...,6.251892,0.000000,0.000000,0.000000,0.000000,0.000000,-2.198108,0.000000,0.000000,0.000000
1,2,100,1.333700,-3.036300,3.613700,1.623700,-5.126300,-12.406300,-3.476300,-8.086300,...,0.073700,-7.696300,-3.036300,5.113700,-2.936300,-4.886300,0.313700,-2.406300,-7.066300,-1.676300
2,3,49,0.000000,0.000000,0.000000,0.000000,1.930612,2.170612,1.930612,2.170612,...,0.000000,0.000000,0.000000,1.980612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4,48,0.000000,5.691875,0.000000,0.000000,-0.858125,5.501875,-5.478125,3.551875,...,0.000000,0.000000,0.000000,-2.128125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,91,5.247363,1.357363,-7.422637,-8.642637,-1.892637,-1.652637,3.787363,1.357363,...,1.937363,2.327363,1.017363,1.937363,2.477363,-1.702637,-0.142637,3.297363,-1.452637,-1.652637


In [25]:
def weight_factor(x, y):
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

In [35]:
ratings_df = pd.read_csv('./Data/structured.csv')
normalized_ratings_df = pd.read_csv('./Data/normalized.csv')

In [36]:
complete_ratings = normalized_ratings_df[normalized_ratings_df['Jokes_Rated'] == 100]
print('total user count who have rated all the jokes: ', len(complete_ratings))
sparse_ratings = normalized_ratings_df[normalized_ratings_df['Jokes_Rated'] != 100]
print('total user count who have not rated all the jokes: ', len(sparse_ratings))

total user count who have rated all the jokes:  14116
total user count who have not rated all the jokes:  59305


In [37]:
n = 1000
active_user_id = sparse_ratings.iloc[n, 0]
print("Let's selct a random user with user id {} as active user for\
 which we will recommend the joke".format(str(active_user_id)))

Let's selct a random user with user id 1352 as active user for which we will recommend the joke


In [39]:
print('ratings given by active user {} for 100 jokes'.format(str(active_user_id)))
active_user = sparse_ratings[sparse_ratings['User_ID'] == active_user_id]
active_user_rating = active_user.iloc[:, 2:]
active_user_rating

ratings given by active user 1352 for 100 jokes


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1351,7.871096,-1.888904,-2.568904,-1.118904,2.821096,-4.078904,-0.188904,1.891096,-2.568904,3.931096,...,0.0,-3.448904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
active_user_rating_list = active_user_rating.values.ravel()
similarity = np.array([(complete_ratings.iloc[i, 0],\
             weight_factor(active_user_rating_list, complete_ratings.iloc[i, 2:]))\
             for i in range(complete_ratings.shape[0])])

In [41]:
ind = np.argsort( similarity[:,1] )
similarity = similarity[ind]
similarity

array([[ 2.76640000e+04, -3.52644866e-01],
       [ 3.26670000e+04, -3.34265224e-01],
       [ 2.53620000e+04, -3.30902199e-01],
       ...,
       [ 1.92170000e+04,  3.99469178e-01],
       [ 1.06150000e+04,  4.06355890e-01],
       [ 7.49200000e+03,  4.59458050e-01]])

In [42]:
neighbours = similarity[similarity[:,1] > 0.1]
print('We have {} potential neighbours! Now we will be randomly selecting 30 samples out of them'.format(len(neighbours)))

We have 4601 potential neighbours! Now we will be randomly selecting 30 samples out of them


In [43]:
index_30_neighbour = np.random.choice(range(len(neighbours)), 30, replace=False)
selected_neighbours = neighbours[index_30_neighbour]
selected_neighbours

array([[2.22370000e+04, 1.85484077e-01],
       [7.85400000e+03, 2.26765811e-01],
       [1.74180000e+04, 1.18470092e-01],
       [4.29530000e+04, 2.12868170e-01],
       [1.71450000e+04, 1.10229387e-01],
       [3.23430000e+04, 1.68173522e-01],
       [1.01690000e+04, 1.03389617e-01],
       [2.43830000e+04, 1.45124215e-01],
       [1.94240000e+04, 1.49006651e-01],
       [1.94760000e+04, 1.52896123e-01],
       [2.05940000e+04, 1.47137133e-01],
       [2.22150000e+04, 2.02288268e-01],
       [2.19490000e+04, 1.38768877e-01],
       [2.77370000e+04, 1.83470206e-01],
       [4.58600000e+03, 1.14545366e-01],
       [1.72250000e+04, 2.18664922e-01],
       [1.64530000e+04, 1.75357933e-01],
       [4.96300000e+03, 1.02114525e-01],
       [2.39240000e+04, 1.58878797e-01],
       [8.64300000e+03, 1.13133065e-01],
       [6.95000000e+02, 1.19769265e-01],
       [4.48230000e+04, 3.11900037e-01],
       [4.57130000e+04, 1.26407130e-01],
       [2.51730000e+04, 2.04802814e-01],
       [2.703700

In [44]:
recommendation_columns = [column for column in active_user_rating.columns if active_user_rating[column].values[0] == 0]

In [46]:
active_user_raw_ratings = ratings_df[ratings_df['User_ID'] == active_user_id].iloc[:, 2:]
active_user_mean_rating = np.mean(active_user_raw_ratings.drop(recommendation_columns, axis = 1).values)
active_user_mean_rating

0.6289041095890411

In [47]:
# Selecting neighbours user_id
neighbour_user_id = selected_neighbours[:, 0]


# selectig neighbour user similarity
neighbour_user_similarity = selected_neighbours[:, 1]

# viewing all neighbours user id and similarity
print('neighbours user id: ', neighbour_user_id, '\n\n')
print('neighbours user similarity: ', neighbour_user_similarity)

neighbours user id:  [22237.  7854. 17418. 42953. 17145. 32343. 10169. 24383. 19424. 19476.
 20594. 22215. 21949. 27737.  4586. 17225. 16453.  4963. 23924.  8643.
   695. 44823. 45713. 25173. 27037.  1964. 44998. 41570. 16446. 13017.] 


neighbours user similarity:  [0.18548408 0.22676581 0.11847009 0.21286817 0.11022939 0.16817352
 0.10338962 0.14512422 0.14900665 0.15289612 0.14713713 0.20228827
 0.13876888 0.18347021 0.11454537 0.21866492 0.17535793 0.10211452
 0.1588788  0.11313307 0.11976927 0.31190004 0.12640713 0.20480281
 0.12808025 0.1080989  0.16240818 0.22199028 0.14128956 0.21746955]


In [49]:
neighbours_df = complete_ratings[complete_ratings['User_ID'].isin(neighbour_user_id)]
len(neighbours_df)

30

In [50]:
# selecting only recommendation columns
print('We will be suggesting one out of {} jokes to the active user \n\n'.format(len(recommendation_columns)))

neighbours_df = neighbours_df[recommendation_columns]
neighbours_df.head()

We will be suggesting one out of 27 jokes to the active user 




,71,72,73,74,75,76,77,78,80,82,...,90,91,93,94,95,96,97,98,99,100
694,-2.6178,3.8822,6.8422,1.8022,-0.8678,7.0922,-1.0678,5.1522,7.5722,-4.4178,...,5.5822,6.8922,-4.1778,1.4622,5.2422,-0.9178,-8.2978,-7.6178,-1.5478,0.0522
1963,0.7112,2.5012,1.1412,-12.7388,-13.5188,2.3112,2.4112,-0.5588,-0.5988,-9.3888,...,2.5512,1.3412,4.4412,3.3812,3.3312,2.6512,3.7712,-2.0588,-0.8888,2.6512
4585,-5.7130,2.5870,-1.7330,-7.9930,-5.1230,0.3070,1.4270,-1.1430,-10.1230,-4.8830,...,-1.1030,1.9070,1.6170,2.1070,1.9570,8.2670,2.7370,1.6670,-4.7430,-1.1030
4962,0.8223,2.4223,-5.0977,-2.7677,-6.5077,-0.9677,0.4323,-1.4577,-4.2777,-0.8777,...,-0.7777,2.2323,9.0323,8.5923,7.0823,5.8223,-3.2077,4.9023,5.1923,-2.6677
7853,-0.3199,-0.7099,-2.4599,-2.6099,-0.7599,0.2601,-2.0199,-0.0799,-0.9099,-3.3299,...,0.4001,1.5701,1.7601,1.8601,0.4501,0.2101,-0.8599,-2.7999,-3.1899,-2.9899


In [51]:
def score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating ):
    item_rating = neighbours_df[item_id]
    t1, t2 = 0, 0
    for similarity, norm_rating in zip(neighbour_user_similarity, item_rating):
        t1+= norm_rating * similarity
        t2+= similarity
    score = (t1 + active_user_mean_rating)/t2
    return score

In [52]:
top_score = -np.inf
joke_to_suggest = ''

for column in neighbours_df.columns:
    score =score_user_item(column, neighbours_df,neighbour_user_similarity, active_user_mean_rating)
    if score > top_score:
        top_score = score
        joke_to_suggest = column
print('highest score is', top_score)
print('The highest score obtained by the joke among all the unseen jokes is', joke_to_suggest, )    

highest score is 3.3332696859618367
The highest score obtained by the joke among all the unseen jokes is 89
